In [199]:
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn import decomposition

In [162]:
def return_list_of_data(bpt = 1000):
    data_dir = 'E:\ELL_Project\problem3\\'
    lis_folders = os.listdir(data_dir+'.')[:-1]
    X = []
    cur_class = 0
    for folder in lis_folders:
        i=0
        lis_files = os.listdir(data_dir+folder+'\.')
        im = []
        for files in lis_files:
            i+=1
            if(i==bpt+1):
                break
            img = '\\'.join([data_dir,folder,files])
            image = plt.imread(img)
            im.append(image.flatten().reshape(-1,1))
        im_np = np.array(im)[:,:,0]
        im_np = np.append(im_np,cur_class*np.ones((im_np.shape[0],1)),axis=1)
        cur_class+=1
        X.append(im_np)
    return X

In [204]:
def dimensional_shrinking_pca(X,n_components=2):
    pca = decomposition.PCA(n_components)
    X_train = X[0]
    for x in X[1:]: 
        X_train = np.append(X_train,x,axis=0)
    X_pca = X_train
    np.random.shuffle(X_pca)
    pca.fit(X_pca[:,:-1])
    X_pca_ret = np.zeros((X_train.shape[0],n_components))
    y_pca_ret = X_pca[:,-1].reshape(-1,1)
    X_pca_ret = pca.transform(X_pca[:,:-1])

    return (X_pca_ret,y_pca_ret)
    

In [205]:
def one_hot(y,num_feats):
    y = y.reshape(-1,1)
    print(y)
    ret = np.zeros((y.shape[0],num_feats))
    rows = np.arange(y.shape[0])
    # print(y)
    # print(rows)
    ret[ rows , y[:,0].astype(int)] = 1
    return ret

In [206]:
def import_dataset(X,y,split_percent = 70):
    
    datasize = X.shape[0]

    split_point = split_percent//10

    X_train = X[:(datasize*split_point)//10,:]
    y_train = y[:(datasize*split_point)//10,:]
    X_test = X[(datasize*split_point)//10:,:]
    y_test = y[(datasize*split_point)//10:,:]

    return(X_train,X_test,y_train,y_test)

In [207]:
def lin_hyp(X,theta):
    return (np.dot(X,theta))

In [208]:
def null_regularizer(alpha,theta):
    a = np.zeros(theta.shape)
    return(a,a)

In [209]:
def l1_reg(alpha,theta):
    reg_loss = alpha*theta
    reg_grad = alpha
    return(reg_loss,reg_grad)

In [210]:
def l2_reg(alpha,theta):
    reg_loss = alpha * np.square(theta)
    reg_grad = 2 * alpha * theta
    return(reg_loss,reg_grad)

In [211]:
def elastic_net_reg(lambda1,lambda2,theta):
    a1,a2 = l1_reg(lambda1,theta)
    b1,b2 = l2_reg(lambda2,theta)
    return( a1+b1 , a2+b2 )

In [212]:
def mse_loss(X,theta,y,hypothesis):
    siz = y.shape[0]
    h = hypothesis(X,theta)
    diff = h-y
    mse = ( np.sum(np.square(diff),axis=0))[0] / siz
    gradient = np.dot(np.transpose(X),diff) / siz

    return(h,mse,gradient)


In [213]:
def mae_loss(X,theta,y,hypothesis):
    siz = y.shape[0]
    num_feat = y.shape[1]

    h = hypothesis(X,theta)
    diff = h-y
    diff_sign = np.ones((siz,1))
    diff_sign[diff[:,0]<0] = -1

    mae = np.sum(np.abs(h),axis=0)[0] / siz
    gradient = np.sum(diff_sign*X,axis=0) / siz

    return(h,mae,gradient)



In [214]:
def ce_loss(X,theta,y,hypothesis):
    siz = y.shape[0]

    h = hypothesis(X,theta)
    h_comp = 1-h
    diff = h - y

    ce = np.sum(-y*log(h)-(1-y)*log(1-h),axis=0) [0] / siz
    gradient = np.dot(np.transpose(X),diff) / siz

    return(h,ce,gradient)   

In [215]:
def gradDesc(X,y,theta,hypothesis,loss_function,regularizer,alpha):
    h,loss,gradient = loss_function(X,theta,y,hypothesis)
    reg_loss,reg_grad = regularizer(alpha,theta)
    loss += reg_loss
    gradient += reg_grad

    return(loss,gradient)

In [216]:
def linReg(X,y,iter=100,alpha=0.01,batchSize=32,n_components=2,classes=6):
    theta = np.random.random((X.shape[1],y.shape[1]))
    datasize = X.shape[0]
    for cl in range(classes):
        y_class = y[:,cl:cl+1]
        loss_epoch,_ = gradDesc(X,y_class,theta[:,cl:cl+1],lin_hyp,mse_loss,l2_reg,alpha)
        for i in range(iter+1):
            if((i)%100000==0):
                print('Loss for {} iterations: {}'.format(i,np.sum(loss_epoch,axis=0)[0]))
            fro = 0 
            loss_epoch = 0
            while(True):
                to = min(fro+batchSize,datasize)
                l,theta_grad = gradDesc(X[fro:to,:],y_class[fro:to,:],theta[:,cl:cl+1],lin_hyp,mse_loss,l2_reg,alpha)
                loss_epoch += l
                theta[:,cl:cl+1] -= (alpha*theta_grad)
                fro = to

                if(to>=datasize):
                    break
    
    return (theta,loss_epoch)

In [217]:
def feature_scaling(X_train):
    training_size = X_train.shape[0]
    X_mean = np.sum(X_train,axis=0) / training_size
    X_var = np.sqrt(np.sum((np.square(X_train-X_mean)),axis=0)/training_size)
    X_train_reg = (X_train - X_mean) / X_var
    return (X_mean,X_var,X_train_reg)

In [218]:
def poly_feat(X,X_var,degree=2):
    num_feats = X.shape[1]
    num_vals = X.shape[0]
    X_prev = X
    X_cross = np.append(np.ones(X.shape),X,axis=1)
    if(degree==0):
        return(np.ones(X.shape))
    if(degree==1):
        return(X_cross)
    for i in range(num_feats):
        for j in range(i+1,num_feats):
            X_cross = np.append( X_cross ,  np.multiply ( X[:,i:i+1] , X[:,j:j+1] ) , axis=1 )
    for i in range(2,degree):
        X_prev = np.multiply(X_prev,X)
        X_prev = X_prev / X_var
        X_cross = np.append(X_cross,X_prev,axis=1)

    

    return ( X_cross )

In [219]:
def polyReg(X,y,iter=100,alpha=0.01,batchSize=32,degree=2,n_classes = 6):
    X_mean,X_var,X_norm = feature_scaling(X)
    X_cross = poly_feat(X_norm,X_var,degree)
    train_size = X_cross.shape[0]
    X_cross = np.append(np.ones((train_size,1)),X_cross,axis=1)
    y_onehot = one_hot(y,n_classes)
    opt, tl = linReg(X_cross,y_onehot,iter=iter,alpha=alpha,batchSize=batchSize)
    return ( X_mean, X_var, opt , tl )


In [220]:
def predicter(X_mean, X_var, X_test, opt_theta,degree=2):
    X_test_norm = (X_test - X_mean) / X_var
    X_test_cross = poly_feat(X_test_norm, X_var,degree)
    test_size = X_test_cross.shape[0]
    X_test_cross = np.append( np.ones((test_size,1)) , X_test_cross , axis=1)

    # print(X_test_cross.shape)
    # print(opt_theta.shape)
    print(degree)
    print(X_test_cross)
    print(opt_theta)
    y_pred = np.dot(X_test_cross, opt_theta)
    y_pred_norm = y_pred / (np.sum(y_pred,axis=1).reshape(-1,1))
    y_pred_class = y_pred.argmax(1)

    return y_pred_class,y_pred_norm


In [221]:
def acc(X_mean,X_var,X_test,y_test,opt_theta,degree=2,n_classes=6):
    y_pred_class,y_pred_norm = predicter(X_mean, X_var, X_test, opt_theta, degree)
    y_test_one_hot = one_hot(y_test,n_classes)
    y_pred_one_hot = one_hot(y_pred_class,n_classes)
    for i in range(n_classes):
        print('class {} .......................................'.format(i))
        accuracy_metrics_classif(y_pred_one_hot[:,i:i+1],y_test_one_hot[:,i:i+1])


def accuracy_metrics_classif(y_pred,y_test):
    
    loss_y = y_pred - y_test

    test_size = y_pred.shape[0]
    total_loss_y = np.dot(np.ones((1,test_size)),np.square(loss_y))[0,0] / test_size

    # print(total_loss_y)

    y_pred_thresh = y_pred>=0.5

    tp = np.sum((y_pred_thresh+y_test)==2 , axis=0)[0]
    tn = np.sum(y_pred_thresh==y_test , axis=0)[0] - tp
    fp = np.sum(y_pred_thresh , axis=0)[0]-tp
    fn = test_size-tp-tn-fp


    print('tp: {} , tn: {} , fp: {} , fn: {}'.format(tp,tn,fp,fn))

    acc = (tp+tn)/test_size
    prec = (tp)/(tp+fp)
    recl = (tp)/(tp+fn)
    f1 = 2*prec*recl/(prec+recl)

    print('Accuracy: {}'.format( acc  ))
    print('Precision: {}'.format( prec  ))
    print('Recall: {}'.format( recl  ))
    print('F1 score: {}'.format( f1  ))




In [222]:
def print_accuracy_metric_classif(X_mean,X_var,X_train,X_test,y_train,opt_theta,degree=2):
    print('Train Accuracy')
    accuracy_metrics_classif(X_mean,X_var,X_train,y_train,opt_theta,degree)

    print('..............................................')

    print('Test Accuracy')
    accuracy_metrics_classif(X_mean,X_var,X_test,y_test,opt_theta,degree)

In [223]:
def calc_loss(y_pred,y_test):
    return(np.sum(np.square(y_pred-y_test),axis=0)[0] / y_pred.shape[0])

In [224]:
def plotter(y_test,y_pred):
    siz = y_test.shape[0]
    x_plot = np.arange(0, siz, 1)
    # print(x_plot[:10])
    plt.plot(x_plot, y_pred, 'g')
    plt.plot(x_plot, y_test, 'b')
    plt.show()

In [225]:
def model_run(X_train,X_test,y_train,y_test,degree = 3, plot = False,alpha=0.0001,n_classes = 6):
    # X_mean,X_var, X_train_feat_scaled = feature_scaling(X_train , X_test)
    train_datasize = X_train.shape[0]
    X_mean, X_var, opt_theta, train_loss = polyReg(X_train,y_train,50000,alpha,train_datasize,degree,n_classes)
    y_pred = predicter(X_mean, X_var, X_test, opt_theta,degree)
    acc(X_mean,X_var,X_test,y_test,opt_theta,degree=2)
    if(plot):
        plotter(y_pred,y_test)
        y_pred_train = predicter(X_mean, X_var, X_train, opt_theta, degree)
        plotter(y_pred_train,y_train)
    return(X_mean, X_var,opt_theta, degree,y_pred)
# print_accuracy_metric_classif(X_mean,X_var,X_train,X_test,y_train,opt_theta,degree=2)

In [226]:
n_components = 2
n_classes = 6
data = return_list_of_data(bpt = 1000)
X_pca_ret,y_pca_ret = dimensional_shrinking_pca(data,n_components)
X_train,X_test,y_train,y_test = import_dataset(X_pca_ret,y_pca_ret)
pow_l,train_l,test_l = [],[],[]
# for power in range(0,10):
#     X_mean, X_var,opt_theta, degree,y_pred = model_run(X_train,X_test,y_train,y_test,power,False)
#     y_pred_train = predicter(X_mean, X_var, X_train, opt_theta, power)
#     l_train = calc_loss(y_pred_train,y_train)
#     y_pred_test = predicter(X_mean, X_var, X_test, opt_theta, power)
#     if power==10:
#         print('CHECK',y_pred_test[:15,:])
#     l_test = calc_loss(y_pred_test,y_test)
#     pow_l.append(power)
#     train_l.append(l_train)
#     test_l.append(l_test)
power=2
X_mean, X_var,opt_theta, degree,y_pred = model_run(X_train,X_test,y_train,y_test,power,False)
# y_pred_train = predicter(X_mean, X_var, X_train, opt_theta, power)
# l_train = calc_loss(y_pred_train,y_train)
# y_pred_test = predicter(X_mean, X_var, X_test, opt_theta, power)
# l_test = calc_loss(y_pred_test,y_test)
# pow_l.append(power)
# train_l.append(l_train)
# test_l.append(l_test)

# plt.plot(pow_l, test_l, 'g')
# plt.plot(pow_l, train_l, 'b')
# plt.show()


[[5.]
 [1.]
 [0.]
 ...
 [2.]
 [3.]
 [4.]]
Loss for 0 iterations: 12.312046344208262
Loss for 0 iterations: 37.463176481791194
Loss for 0 iterations: 12.628114727056596
Loss for 0 iterations: 6.48036816477775
Loss for 0 iterations: 16.988981117761362
Loss for 0 iterations: 6.838157167178759
2
[[ 1.          1.          1.          0.38874609 -0.75534752 -0.29363839]
 [ 1.          1.          1.          1.32369848  0.58061591  0.7685604 ]
 [ 1.          1.          1.          0.43280281 -0.89736802 -0.3883834 ]
 ...
 [ 1.          1.          1.         -0.68517883  2.01547844 -1.38096316]
 [ 1.          1.          1.         -0.90286907  0.33883577 -0.30592434]
 [ 1.          1.          1.         -0.88429459  0.11863847 -0.10491136]]
[[-0.11539099  0.09671404 -0.06747165 -0.25347331 -0.38768052 -0.25231554]
 [-0.30508943 -0.06676747  0.14636532  0.04273028  0.11674201 -0.08927983]
 [ 0.58542359  0.13263856  0.09041844  0.38196717  0.43866276  0.50639272]
 [ 0.06964125 -0.21441309 

In [227]:
# plt.plot(pow_l, test_l, 'g^')
# plt.plot(pow_l, train_l, 'bs')
# plt.show()

In [ ]:
`